In [1]:
############### packages ##############
#######################################
rom sage.libs.singular.function import singular_function, lib as singular_lib
singular_lib('sing.lib')
import sympy as sp
import collections


############### function ##############
#######################################
def monomials_list(f):
    '''
    Retrun a list consists of all  monomials of f
    
    Input parameter :
      f : a polynomial 

    Return : a list consists of all  monomials of f 

    Example : 
    >>> monomials_list(3*x**2-x*y*2+z**3*x) 
    [Poly(3*x**2, x, domain='ZZ'), Poly(-2*x*y, x, y, domain='ZZ'), Poly(x*z**3, x, z, domain='ZZ')]
    '''
    f = sp.Poly(f)
    return [sp.Poly(c*sp.prod(x**k for x,k in zip(f.gens, mon))) for c, mon in zip(f.coeffs(), f.monoms())]


def homogeneous_part(f, n):
    '''
    Input parameter :
      f : a polynomial 
      n : an integer

    Return : 
      part : the homogeneous part of degree n in f

    Example : 
    >>> homogeneous_part(3*x**2-x*y*2+z**3*x,2) 
    3*x**2 - 2*x*y
    '''
    part = 0
    for term in monomials_list(sp.Poly(f)):
        if sp.Poly(term).total_degree()==n:
            part+=term
    return sp.simplify(part).as_expr()

def jet(f, n):
    '''
    Input parameter :
      f : a polynomial 
      n : an integer

    Return : 
      part : the part of f has degree no more than n

    Example : 
    >>> jet(3*x**2-x*y*2+z**3*x,2) 
    3*x**2-x*y*2
    '''
    part = 0
    for term in monomials_list(sp.Poly(f)):
        if sp.Poly(term).total_degree()<=n:
            part+=term
    return sp.simplify(part).as_expr()

def multiplicity(f):
    '''
    Input parameter :
      f : a polynomial 

    Return : 
      m : the multiplicity of f at origin where m = -1 if f = 0

    Example : 
    >>> multiplicity(3*x**2-x*y*2+z**3*x) 
    2
    '''
    if f == 0:
        return -1
    else:
        return min([sp.Poly(term).total_degree() for term in monomials_list(sp.Poly(f))])

def change_of_var(f, change_list):
    '''
    make a change of variables from var into new_var 
    for any (var, new_var) in change_list
    
    Input parameter :
      f : a polynomial
      change_list : [(var, new_var), ..., (var, new_var)]

    Return : f

    Example : 
    >>>change_of_var(sp.poly('3*x-2*y**2'),[(x, y-1), (y, x+1)])
    3*y - 2*(x + 1)**2 - 3
    '''
    n = len(change_list)
    var = [element[0] for element in change_list]
    new_var = [element[1] for element in change_list]
    replace_var = list(sp.symbols('u0:{}'.format(n)))

    for i in range(n):
        f = f.subs(var[i], replace_var[i])
    for i in range(n):
        f = f.subs(replace_var[i], new_var[i])
    return sp.simplify(sp.expand(f.as_expr()))   
  
def Hessian_matrix(f):
    '''
    Input parameter :
      f : a polynomial

    Return : 
      Hess : the  Hessian matrix of f at origin

    Example : 
    >>> Hessian_matrix(sp.Poly('x**4 + 3*x**2 + 3*x*y + y**5 - 2*y**2 + z**6 - 4*z**3'))
    Matrix([[6, 3, 0], [3, -4, 0], [0, 0, 0]])
    '''
    var_list = f.gens
    Hess = sp.hessian(f, var_list, constraints=[])
    for i in var_list:
        Hess = Hess.subs(i,0)

    return Hess


def corank_Hessian_matrix(f):
    '''
    Input parameter :
      f : a polynomial

    Return : the corank of f's Hessian matrix at origin

    Example : 
    >>> corank_Hessian_Matrix(sp.Poly('x**4 + 3*x**2 + 3*x*y + y**5 - 2*y**2 + z**6 - 4*z**3'))
    1
    '''
    var_list=f.gens
    return len(var_list)-sp.Matrix.rank(Hessian_matrix(f))

def quadratic_form(f):
    '''
    Find out the symmetry matrix 
    corresponds to the quadratic form of degree 2 part of f.
    
    Input parameter :
      f : a polynomial

    Return : 
      Q : the symmetry matrix

    Example : 
    >>> quadratic_form(sp.Poly('x**4 + 3*x**2 + 3*x*y + y**5 - 2*y**2 + z**6 - 4*z**3'))
    Matrix([[  6, 3/2, 0],[3/2,  -4, 0],[  0,   0, 0]])
    '''
    Q = Hessian_matrix(f)/2
    return Q

def rearrange(v_list): 
    '''
    Input parameter :
      v_list : a list of some integer
      
    Return : 
      v : rearrange v_list such that all zeros are in the tail 

    Example : 
    >>> rearrange([1,0,-1,0,2])
    [2,1,-1,0,0]]
    '''
    v0 = [x for x in v_list if x==0]    
    v = sorted([x for x in v_list if x!=0])
    v.extend(v0)        
    return v

def quadratic_transform(f):
    '''
    Make a change of variables such that the degree 2 part is
    a diagonal form whose coefficients are all one 
    
    Input parameter :
      f : a polynomial
      
    Return : 
      f_new : a polynomial

    Example : 
    >>>quadratic_transform(sp.Poly('x**4 + x**2 + 3*x*y + z**5 + y**2 '))
    x**4 - 4*sqrt(5)*I*x**3*y/5 - 6*x**2*y**2/5 + 4*sqrt(5)*I*x*y**3/25 + y**4/25 + x**2 + y**2
    '''    
    Q = quadratic_form(f)
    v_eigenvec = Q.eigenvects()
    v_eigenvec = [list(x) for x in list(v_eigenvec)]

    eigenvalue0 = [x[0] for x in v_eigenvec]
    eigenvalue = rearrange(eigenvalue0)
    
    
    eigenv = []
    M = sp.Matrix()
    for i in eigenvalue:
        index = eigenvalue0.index(i)
        for vec in v_eigenvec[index][2]:
            eigenv.append(i)
            M = M.row_join(vec.normalized())
            
    diag = [1/sp.sqrt(x) if x != 0 else 1 for x in eigenv]
    D = sp.diag(*diag)
    #print(Q , D*M.T*Q*M*D)
    
    var = sp.Matrix(f.gens)
    new_var = (M*D)*var
    change = [(var[i],new_var[i]) for i in range(len(var))]
    f_new = sp.simplify(change_of_var(f, change))
    
    return f_new


def complete_the_square(f, var):
    '''
    Complete the square f = a*(var-terms)**2 + g
    and make a change of variable
    such that f = var**2+new_g
    
    Input parameter :
      f : a polynomial
      var : a variable in f

    Return : new_f
      
    Example : 
    >>> complete_the_square(3*x**2+3*x*y-2*y*2-4*z*3, x)
    x**2 - 3*y**2/4 - 4*y - 12*z
    '''
    f_part = f - var**2 
    f_part_remainder = f_part.subs({var:0})
    f_part = f_part - f_part_remainder
    
    if f_part == 0:
        return f
    
    elif len(sp.Poly(f_part).gens) == 1:
        return f_part_remainder + var**2
    
    else:
        terms = sp.quo(f_part, var)/2
        f = f.subs(var, var-terms)
        f = f.expand()

        return sp.simplify(f).as_expr()

def complete_the_square_new(f, var, determinacy):
    '''
    Complete the square f = a*(var-terms)**2 + g with using finite determinacy theorem
    and make a change of variable
    such that f = var**2+new_g
    
    Input parameter :
      f : a polynomial
      var : a variable in f

    Return : new_f
      
    Example : 
    >>> complete_the_square(3*x**2+3*x*y-2*y*2-4*z*3, x)
    x**2 - 3*y**2/4 - 4*y - 12*z
    '''
    f_part = f - var**2 
    f_part_remainder = f_part.subs({var:0})
    f_part = f_part - f_part_remainder
    
    if f_part == 0:
        return f
    
    elif len(sp.Poly(f_part).gens) == 1:
        return f_part_remainder + var**2
    
    else:
        terms = sp.quo(f_part, var)/2
        f = f.subs(var, jet(var-terms, determinacy))
        f = jet(f.expand(), determinacy)

        return sp.simplify(f).as_expr()


def factor(f):
    ''' 
    factor the homogeneous polynomial
    and return all factors of it
    
    Input parameter :
      f : a polynomial

    Return : 
      factor_list : consists of all factors of f
      
    Example : 
    >>> factor(y*z**2+y**2*z+y**3)
    [y - z*(-1/2 - sqrt(3)*I/2), y - z*(-1/2 + sqrt(3)*I/2), y]
    '''
    f = sp.Poly(f) 
    var_list=f.gens
    factor_list = []
    for x in var_list:
        dic_roots = sp.roots(f, x)
    
        for a in dic_roots:
            for i  in range(dic_roots[a]):
                factor_list.append(sp.simplify(x-a))
                f = sp.quo(f,x-a)
    return factor_list

def find_linear_factor(f, n):
    ''' 
    find the linear factors of the degree n homogeneous part of f 
    
    Input parameter :
      f : a polynomial
      n : an integer

    Return : 
      num_roots : the number of the different linear factors
      factor_dic : the dictionary of all linear factors with multiplicity as value (type: collections.Counter)
      
    Examples : 
    >>> find_linear_factor(sp.Poly('y*z**2+y**2*z+y**3'), 2)
    (0, {})
    >>> find_linear_factor(sp.Poly('y*z**2+y**2*z+y**3'), 3)
    (3, Counter({y: 1, y - z*(-1/2 + sqrt(3)*I/2): 1, y - z*(-1/2 - sqrt(3)*I/2): 1}))
    '''
    f = homogeneous_part(f, n)
    if f == 0:
        num_roots, factor_dic = 0, {}
    else:
        factor_list = factor(f)
        factor_dic = collections.Counter(factor_list)
        num_roots = len(factor_dic)
        
    return num_roots

def cubic_form_lemma(f):
    ''' 
    determine  whether the cubic form f can be divisible by a square of a linear form or not
    
    Input parameter :
      f : a cubic form

    Return : 
        bool: True or False
        integer: the largest number of main linear factors
        polynomial: main linear factor
        polynomial: another linear factor
      
    Examples : 
    '''
    f = sp.Poly(f) 
    var_list = f.gens
    
    assert len(var_list)<=3 and f.is_homogeneous, 'The input must be a cubic form!'
    
    partial_list = []
    for var in var_list:
        partial_list.append(f.diff(var))
    gcd = sp.gcd(partial_list)
    gcd_deg = gcd.total_degree()
    
    if gcd_deg > 0 and f.rem(gcd) == 0:
        if gcd_deg == 2:
            for var in var_list:
                gcd_diff = gcd.diff(var)
                if gcd_diff != 0:
                    return True, 3, gcd_diff/6, 0
                else:
                    pass
        else:
            l2 = f.quo(gcd**2)
            return True, 2, gcd, l2 
    else:
        return False, 0, 0, 0

    
def group_action_condition(singularity_type, index, group_action):
    if singularity_type == 'cA':
        if index == 2 and group_action == [0, 1, 1, 1]:
            return 'cAx/2'
        elif index == 4 and group_action == [1, 3, 1, 2]:
            return 'cAx/4'
        else:
            r = group_action[0]
            if gcd(index, r) == 1 and group_action == [r, index-r, 1, 0]:
                return 'cA/'+str(index)
            else:
                return 'non-terminal'            
    
    elif singularity_type == 'cD':
        if index == 2 and group_action == [1, 0, 1, 1]:
            return 'cD/2'
        elif index == 3 and group_action == [0, 2, 1, 1]:
            return 'cD/3'
        else:
            return 'non-terminal'  
    
    elif singularity_type == 'cE':
        if index == 2 and group_action == [1, 0, 1, 1]:
            return 'cE/2'
        else:
            return 'non-terminal'         
    
    else:
        return 'non-terminal'
    
    
############ main function ############
#######################################
def terminal_singularity_with_index_1(f0):
    R = singular.ring(singular.complex, '(x,y,z,u)', 'ds')
    f_singular = singular(f0)
    var_list = sp.Poly(f0).gens
    
    f_hessian_singular = singular.jacob(singular.jacob(f_singular.jet(2)))
    f_rank =  f_hessian_singular.rank()
    f_corank = f_hessian_singular.nrows() - f_rank

    if f_rank >= 2:
        ## compute an upper bound of determinacy, i.e. estimated determinacy
        j = singular.jacob(f_singular)
        j = singular.std(j)
        f_determinacy = singular.deg(singular.highcorner(j)) + 2

        #####------splitting_lemma-------#####
        f_i = quadratic_transform(sp.Poly(f0))
        for i in range(3, f_determinacy):
            f_i0 = f_i.copy() 

            for j in range(int(f_rank)):
                f_i = complete_the_square(f_i, var_list[j])

            if f_i == f_i0:
                break
        
        for j in range(int(f_rank)):
            f_i -= var_list[j]**2
        n = multiplicity(f_i)
        return 'cA_%s' %n

    elif f_rank == 1:
        ## compute an upper bound of determinacy, i.e. estimated determinacy
        j = singular.jacob(f_singular)
        j = singular.std(j)
        f_determinacy = singular.deg(singular.highcorner(j)) + 2

        #####------splitting_lemma-------#####
        f_i = quadratic_transform(sp.Poly(f0))
        for i in range(3, f_determinacy):
            f_i0 = f_i.copy() 

            for j in range(int(f_rank)):
                f_i = complete_the_square(f_i, var_list[j])

            if f_i == f_i0:
                break
        
        
        f_3_part = homogeneous_part(f_i, 3)

        if cubic_form_lemma(f_3_part)[0]:
            if cubic_form_lemma(f_3_part)[1] == 2:
                return 'cD>4'

            else:

                L1 = cubic_form_lemma(f_3_part)[2]

                # Row 1
                r1 = [L1.subs({var_list[1]:1,var_list[2]:0,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:1,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:0,var_list[3]:1})]

                linear_trans = sp.Matrix([r1])

                # choose standard basis
                I = sp.eye(3)
                for i in range(3):
                    if linear_trans.dot(I[i,:]) != 0:
                        j = i
                        break

                for i in range(3):
                    if i!=j:
                        linear_trans = linear_trans.col_join(I[i,:])

                linear_factor = list(linear_trans.inv()*sp.Matrix([[var_list[1]],[var_list[2]],[var_list[3]]]))
                f = change_of_var(f_i, [(var_list[1], linear_factor[0]), (var_list[2], linear_factor[1]), (var_list[3], linear_factor[2])])


                # f ~ x^2 + y^3 + y^2*g_1(y,z,u) + y*g_2(z,u) + g_3(z,u)
                f = jet(f, f_determinacy)
                for i in range(2):
                    g_1 = sp.quo(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)

                    if g_1 == 0:
                        break
                    else:
                        f = change_of_var(f, [(var_list[1], var_list[1] - g_1/3)])
                        f = jet(f, f_determinacy)
                        
                # f_4 = y*g(z,u) + h(z,u)
                f_4 = sp.rem(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)
                g = sp.quo(f_4, var_list[1])
                h = sp.rem(f_4, var_list[1])
                
                mult_g = multiplicity(g)
                mult_h = multiplicity(h)
                
                if mult_h == 4:
                    return 'cE_6'
                
                elif mult_g == 3:
                    return 'cE_7'
                
                elif mult_h == 5:
                    return 'cE_8'
                    
                else:
                    return 'non-terminal'

        else:
            return 'cD_4.'

    else:
        return 'non-terminal'

def terminal_singularity_with_index_more_than_1(f0, index, group_action):
    R = singular.ring(singular.complex, '(x,y,z,u)', 'ds')
    f_singular = singular(f0)
    var_list = sp.Poly(f0).gens
    
    f_hessian_singular = singular.jacob(singular.jacob(f_singular.jet(2)))
    f_rank =  f_hessian_singular.rank()
    f_corank = f_hessian_singular.nrows() - f_rank

    if f_rank >= 2:
        return group_action_condition('cA', index, group_action)

    elif f_rank == 1:
        ## compute an upper bound of determinacy, i.e. estimated determinacy
        j = singular.jacob(f_singular)
        j = singular.std(j)
        f_determinacy = singular.deg(singular.highcorner(j)) + 2

        #####------splitting_lemma-------#####
        f_i = quadratic_transform(sp.Poly(f0))
        for i in range(3, f_determinacy):
            f_i0 = f_i.copy() 

            for j in range(int(f_rank)):
                f_i = complete_the_square(f_i, var_list[j])

            if f_i == f_i0:
                break
        
        
        f_3_part = homogeneous_part(f_i, 3)

        if cubic_form_lemma(f_3_part)[0]:
            if cubic_form_lemma(f_3_part)[1] == 2:
                return group_action_condition('cD', index, group_action)

            else:

                L1 = cubic_form_lemma(f_3_part)[2]

                # Row 1
                r1 = [L1.subs({var_list[1]:1,var_list[2]:0,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:1,var_list[3]:0}),
                                           L1.subs({var_list[1]:0,var_list[2]:0,var_list[3]:1})]

                linear_trans = sp.Matrix([r1])

                # choose standard basis
                I = sp.eye(3)
                for i in range(3):
                    if linear_trans.dot(I[i,:]) != 0:
                        j = i
                        break

                for i in range(3):
                    if i!=j:
                        linear_trans = linear_trans.col_join(I[i,:])

                linear_factor = list(linear_trans.inv()*sp.Matrix([[var_list[1]],[var_list[2]],[var_list[3]]]))
                f = change_of_var(f_i, [(var_list[1], linear_factor[0]), (var_list[2], linear_factor[1]), (var_list[3], linear_factor[2])])


                # f ~ x^2 + y^3 + y^2*g_1(y,z,u) + y*g_2(z,u) + g_3(z,u)
                f = jet(f, f_determinacy)
                for i in range(2):
                    g_1 = sp.quo(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)

                    if g_1 == 0:
                        break
                    else:
                        f = change_of_var(f, [(var_list[1], var_list[1] - g_1/3)])
                        f = jet(f, f_determinacy)
                        
                # f_4 = y*g(z,u) + h(z,u)
                f_4 = sp.rem(f - var_list[0]**2 - var_list[1]**3, var_list[1]**2)
                
                g = sp.quo(f_4, var_list[1])
                h = sp.rem(f_4, var_list[1])

                mult_g = multiplicity(g)
                mult_h = multiplicity(h)
                
                if mult_h == 4 or 5 and mult_g == 3:
                    return group_action_condition('cE', index, group_action)
                    
                else:
                    return 'non-terminal'

        else:
            return group_action_condition('cD', index, group_action)

    else:
        return 'non-terminal'